In [ ]:
import docx2txt
import subprocess
from subprocess import STDOUT,PIPE
from time import sleep
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
driver = webdriver.Chrome(executable_path="../drivers/chromedriver")
import docx

In [ ]:
default = "(base) harshitsharma@192\n"
solutions = []

def compile_java(java_file):
    try:
        subprocess.check_call(['javac', java_file])
    except subprocess.CalledProcessError as e:
        print(e.output)
    

def execute_java(java_file, java_class, inputi):
    cmd = ['java', java_class]
    proc = subprocess.run(cmd,stdout=PIPE,stderr=STDOUT, input=inputi.encode())
    if proc.stderr is not None:
        solutions.append("PLEASE NOTE THAT THIS CODE DIDNT WORKED. PLEASE CHECK THE CODE, EDIT IT MANUALLY OR TRY RUNNING THIS SCRIPT AGAIN. ERROR = "+ proc.stderr.decode())
    else:
        solutions.append(proc.stdout.decode())

In [ ]:
doc = docx2txt.process('files/1.docx')
ans = doc.split("Code")
ques = doc.split("~")

sol = []
inputs = []
inputs = doc.split("Input:")
inputs = inputs[1:]
finalQues = []

for i in range(0, len(inputs)):
    inputs[i] = inputs[i].split('$')[0][1:]

for i in range(0,len(ans)):
    startIndex = ans[i].find("Code")
    endIndex = ans[i].find("Output")
    startIndex +=4
    sol.append(ans[i][startIndex:endIndex])

for i in range(0, len(ques)):
    if(i%2!=0):
        finalQues.append(ques[i])
        


# print("Inputs")
# print(inputs)

# print(finalQues)

sol = sol[1:]

print(sol)
        

In [ ]:
# list = os.listdir('Files')

# for p in range(0, len(list)):

for i in range(0,len(sol)):
    className = sol[i].split("{")[0].split("class")[1].replace(" ","")
    filename = className + ".java"
    subprocess.check_call(["touch",filename])
    subprocess.run([ 'chmod',  '+x',  filename ])
    f = open(filename, "w")
    f.write(sol[i])
    f.close()
    compile_java(filename)
    execute_java(filename, className, inputs[i])
    
for i in range(0, len(solutions)):
    driver.get("https://ray.so/#language=shell&darkMode=true&background=false&padding=16&theme=falcon&title=Terminal&code=")
    textbox = driver.find_element(By.CLASS_NAME, "Editor_textarea__sAyL_")
    textbox.clear()
    input_modifier = inputs[i] + '\n' + solutions[i]
    textbox.send_keys(default + input_modifier)
    sleep(2)
    checkbox = driver.find_element(By.CLASS_NAME, "ExportButton_button__d___t")
    checkbox.click()
    sleep(5)
    
doc = docx.Document()
print(len(finalQues))

doc.add_heading("Lab" + list[i])

for i in range(0, len(finalQues)):
    doc.add_heading("Question "+ str(i+1),)
    doc.add_paragraph(finalQues[i] + '\n')
    doc.add_heading("Code \n")
    doc.add_paragraph(sol[i] + '\n')
    doc.add_heading("Output \n")
    doc.add_picture('../../Downloads/ray-so-export ({}).png'.format(i+1))
    doc.save("Java_Assignment.docx")
